In [1]:
#TODO: add shell commands

In [1]:
#imports
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from kafka import KafkaProducer
import json

In [2]:
#set parameters
source_url = "https://archive-api.open-meteo.com/v1/archive"
kafka_server = 'localhost:9092'
api_params = {
	"latitude": 48.210033,
	"longitude": 16.363449,
	"start_date": "2023-12-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m"]
}

In [7]:
#set up api
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

#get api data and parse into variables
responses = openmeteo.weather_api(source_url, params=api_params)
response = responses[0]
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()

#create dataframe
hourly_data = {"time": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_dataframe = pd.DataFrame(data = hourly_data)
#coonvert timestamps into correct format
hourly_dataframe["time"]=hourly_dataframe["time"].dt.strftime('%d/%m/%y %H:%M:%S.%f')
hourly_dataframe

,time,temperature_2m,relative_humidity_2m,wind_speed_10m
0,01/12/23 00:00:00.000000,0.0235,94.346542,5.600286
1,01/12/23 01:00:00.000000,0.5235,90.332314,7.895416
2,01/12/23 02:00:00.000000,0.5235,90.332314,6.369050
3,01/12/23 03:00:00.000000,0.5235,90.332314,4.510787
4,01/12/23 04:00:00.000000,0.4735,91.992157,3.319036
...,...,...,...,...
739,31/12/23 19:00:00.000000,0.3235,99.638947,2.545584
740,31/12/23 20:00:00.000000,0.1735,99.638504,4.829907
741,31/12/23 21:00:00.000000,0.2235,99.638657,4.582052
742,31/12/23 22:00:00.000000,1.5735,97.172493,13.281023


In [4]:
#init producer
producer = KafkaProducer(bootstrap_servers=kafka_server,value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [5]:
#produce df to kafka row by row
for index, row in hourly_dataframe.iterrows():
    producer.send('weather', row.to_dict())
    producer.flush()

In [6]:
producer.close()